In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import torch 
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torchvision
from torchvision import transforms
import torchvision.models as models

import tqdm

import sys
import os
import ssl
import time
import psutil
ssl._create_default_https_context = ssl._create_unverified_context
print(os.getcwd())
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
print(device)
mem = psutil.virtual_memory()

/Users/kazesword/Documents/5243/Project3/doc/peng_code
cpu


/opt/homebrew/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Get time consumption
def getTime(func):
    def _wrapper(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        print(f"Total time consumption:{end-start:.3f}s")
        return res
    return _wrapper

In [3]:
def image_prepocessing(image,transform):
    return torch.unsqueeze(transform(image.astype(np.uint8)),0).float()

In [5]:
# Using Pre-trained ResNet as image encoder 
img_encoder = models.resnet18(pretrained=True) 
img_encoder.fc = nn.Identity()
img_encoder.to(device).eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
# Image Augmentation
transform_model = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((227,227)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )
])

In [ ]:
@getTime
def encoding(encoder):
    with torch.no_grad():
        encoded_imgs = np.empty((n_img,512))
        for i in tqdm.tqdm(range(n_img)):
            encoded_imgs[i] = encoder(image_prepocessing(imgs[i],transform_model))
            
    # with torch.no_grad():
    #     encoded_imgs = np.empty((X_train.shape[0],512))
    #     for i in tqdm.tqdm(range(X_train.shape[0])):
    #         encoded_imgs[i] = img_encoder(image_prepocessing(X_train[i],transform_model2))
encoding(img_encoder)

In [ ]:
np.savetxt('encoded_imgs.txt', encoded_imgs)